### Importations des bibliothèques

In [ ]:
!pip install numpy==1.21.4
!pip install pandas==1.3.4
import numpy as np  
import pandas as pd
from datetime import datetime
import geopy.distance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.7 MB 115 kB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.3 MB 3.6 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5


### Fonctions

In [ ]:
#calculer la durée entre deux dates
def TimeDiff(date_1,date_2):
  date_format_str = '%Y-%m-%d %H:%M:%S+%f'
  start = datetime.strptime(date_1, date_format_str)
  end =   datetime.strptime(date_2, date_format_str)
  diff = end - start
  diff_in_hours = diff.total_seconds() / 3600
  return(diff_in_hours)

In [ ]:
#calculer la distance entre deux coordonnées
def distances(coords_1, coords_2):
  return(geopy.distance.geodesic(coords_1, coords_2).km)

In [ ]:
#ajouter la colonne de dureé de trajet dans le dataframe
def add_duration(df):
  duration=[]
  for i,trial in df.iterrows():
    duration.append(TimeDiff(df.loc[i, "departure_ts"],df.loc[i, "arrival_ts"]))
  df['duration']=duration
  return(df)

In [ ]:
#ajouter la colonne des trajet dans le dataframe
def add_traget(df):
  traget=[]
  for i,trial in df.iterrows():
    traget.append(df.loc[i, "o"]+' to '+df.loc[i, "d"])
  df['traget']=traget
  return(df)


In [ ]:
#ajouter la colonne de distance du chemin dans le dataframe
def add_distance(df):
  dis=[]
  for i,trial in df.iterrows():
    dis.append(distances(((df.loc[i, "latitude_depart"]),(df.loc[i, "longitude_depart"])),
    ((df.loc[i, "latitude_destination"]),(df.loc[i, "longitude_destination"]))))
  df['distance']=dis
  return(df)

In [ ]:
#suppression de l'anomalie dans la colonne de la durée 
def removing_Outlier(df):
  l=list(df['duration'].quantile([0.25,0.5,0.75]))
  IQR = l[2] -l[0]
  Upper_Bound=(l[2] + 1.5 * IQR)
  def keep(dis):
    if dis <= Upper_Bound :
      return(True)
    else:
      return(False)
  df['keep'] = df['duration'].apply(keep)
  df= df[df.keep == True]
  return(df)


In [ ]:
#Ajout d'une colonne de plage des distances dans le data frame
def add_RangeDistance(df):
  l=list(df['distance'].quantile([0.25,0.5,0.75]))
  def RangeDistance(dis):
    if 0 <= dis and dis <= l[0]:
      return('0-206km')
    elif dis <= l[1]:
      return('207-339km')
    elif dis <= l[2]:
      return('340-481km')
    else:
      return('481+km')
  df['RangeDistance'] = df['distance'].apply(RangeDistance)
  return(df)

### Importation des données

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

MessageError: ignored

In [ ]:
#cities = pd.read_csv("/content/drive/MyDrive/test pfe/cities.csv")
#providers = pd.read_csv("/content/drive/MyDrive/test pfe/providers.csv")
#stations = pd.read_csv("/content/drive/MyDrive/test pfe/stations.csv")
#tickets = pd.read_csv("/content/drive/MyDrive/test pfe/ticket_data.csv")

In [ ]:
cities = pd.read_csv("/content/cities.csv")
providers = pd.read_csv("/content/providers.csv")
stations = pd.read_csv("/content/stations.csv")
tickets = pd.read_csv("/content/drive/MyDrive/test pfe/ticket_data.csv")

In [ ]:
# Aperçu de l'ensemble des données relatives aux villes 
display(cities)
#Les statistiques descriptives correspondantes
cities.describe()

In [ ]:
#Aperçu de l'ensemble des données relatives aux fournisseurs
display(providers)
#Les statistiques descriptives correspondantes
providers.describe()

In [ ]:
# Aperçu de l'ensemble des données relatives aux stations
display(stations)
# Les statistiques descriptives correspondantes
stations.describe()

In [ ]:
# Aperçu de l'ensemble des données relatives aux tickets
display(tickets)
# Les statistiques descriptives correspondantes
tickets.describe()

### Extraction d'informations intéressantes et visualisation


1.   Joindre et fusionner les données nécessaires provenant de différents data frames



In [ ]:
# Conserver les données nécessaires de l'ensemble des données des tickets
All_data=tickets[['id','company','price_in_cents', 'o_city','d_city' ,'o_station','d_station','departure_ts','arrival_ts']]

In [ ]:
# Ajout du nom des villes de départ dans l'ensemble de données
All_data=pd.merge(All_data, cities[['id','unique_name']], left_on='o_city', right_on='id',how='left')
All_data.rename(columns={'unique_name':'o'},inplace=True)
All_data=All_data.drop(['id_y'],axis=1)
# Ajout du nom des villes d'arrivé dans l'ensemble de données
All_data=pd.merge(All_data, cities[['id','unique_name']], left_on='d_city', right_on='id',how='left')
All_data.rename(columns={'unique_name':'d'},inplace=True)
All_data=All_data.drop(['id'],axis=1)
display(All_data)

In [ ]:
# Ajout des coordonnées des villes dans le data frame 
All_data=pd.merge(All_data, cities[['latitude','longitude','unique_name']], left_on='o', right_on='unique_name',how='left')
All_data.rename(columns={'latitude':'latitude_depart','longitude':'longitude_depart'},inplace=True)
All_data=pd.merge(All_data, cities[['latitude','longitude','unique_name']], left_on='d', right_on='unique_name',how='left')
All_data.rename(columns={'latitude':'latitude_destination','longitude':'longitude_destination'},inplace=True)

In [ ]:
# Ajout de compagnies de voyage et de leurs offres le data frame 
All_data=pd.merge(All_data, providers[['id','transport_type','has_bicycle','has_adjustable_seats','has_plug','has_wifi','fullname']], left_on='company', right_on='id',how='left')
display(All_data)

2.    Ajout des colonnes "traget","distance","duration" et"RangeDistance"


 


In [ ]:
All_data=add_traget(All_data)
All_data=add_distance(All_data)
All_data=add_duration(All_data)
All_data=add_RangeDistance(All_data)
#keeping necessary informations
All_data=All_data[['id_x','o', 'd','company', 'price_in_cents','transport_type',
       'has_bicycle', 'has_adjustable_seats', 'has_plug', 'has_wifi',
       'fullname', 'traget', 'distance', 'duration','RangeDistance','o_city','d_city']]
display(All_data)

3.   Visualisation des prix et des durées selon les différents types de transport


In [ ]:
ax=All_data.boxplot(by=['transport_type'],column=['price_in_cents'],figsize =(15, 15))
ax=All_data.boxplot(by=['transport_type'],column=['duration'],figsize =(15, 15))


5.   Élimination des données inadéquates
 

In [ ]:
All_data=removing_Outlier(All_data)
display(All_data)
# l'exportation de données qui seront utilisées ultérieurement dans le modèle de prédiction des prix
All_data[['company', 'price_in_cents','transport_type',
       'has_bicycle', 'has_adjustable_seats', 'has_plug', 'has_wifi', 'distance', 'duration','o_city','d_city']].to_csv('All_data.csv')

6.   Visualisation des prix et des durées selon les différents types de transport après élimination des données inadéquates


In [ ]:
ax=All_data.boxplot(by=['transport_type'],column=['price_in_cents'],figsize =(15, 15))
ax=All_data.boxplot(by=['transport_type'],column=['duration'],figsize =(15, 15))


7.   Visualisation des prix et des durées en fonction des différents types de transport et des différentes sociétés de transport.


In [ ]:
All_data.plot.scatter(x='transport_type', y='fullname', c='price_in_cents',cmap='viridis', s=100,figsize =(10, 10)) 
All_data.plot.scatter(x='transport_type', y='fullname', c='duration',cmap='viridis', s=100,figsize =(10, 10)) 


### Présentation des résultats



1. Presentation des prix min, moyen ,max et median par trajet et durée min, moyenne ,max et mediane par trajet



In [ ]:
results1=All_data.groupby('traget').aggregate({'duration':['min','max','mean','median'],'price_in_cents':['min','max','mean','median']})
display(results1)

2.   Présentation des différents prix moyens, min, max et médians et des temps moyens, min, max et médians selon les différents types de transport : train, bus et covoiturage et selon la distance du trajet : (0-206km),(207-339km),(340-481km) et (481+km) .

In [ ]:
results2=All_data.groupby(['RangeDistance','transport_type']).aggregate({'duration':['min','max','mean','median'],'price_in_cents':['min','max','mean','median']})
display(results2)